In [8]:
import os
import pymysql as sql

sl_no = 1

def g_data_to_csv(filename):
    global sl_no
    
    f = open(r"C:\Users\Admin\Desktop\PROJECT\Python\sqlworkbench\%s"%(filename), 'r')
    content = f.readlines()
    f.close()

    g_symbol = content[0].split()[0].strip()
    g_id = content[1].split(",")[0].split(":")[1].strip()

    g_l = [i for i in content if "primary" in i.lower() and "assembly" in i.lower() and "NC_" in i 
           and "strand" in i.lower() and "from" in i.lower() and "to" in i.lower()][0].strip()

    strand = g_l.split("(")[1].split(")")[0].replace("strand","").strip()
    g_st,g_ed = [int(i.strip()) for i in g_l.split("from:")[1].split("to:")]
    print (g_symbol, g_id, strand, g_st, g_ed)

    l_ind = [i for i in range(len(content)) if "mRNA transcript" in content[i] and "NM_" in 
             content[i]]

    def get_transdata(data_content, trans_id):
        indx = [i for i in range(len(data_content)) if "---------------------------" in 
                data_content[i]][0]
        #['tr,exon_no,ex_st,ex_ed,ex_sz',]
        result = []
        data = [i.strip().split("\t")[0] for i in data_content[indx + 1:] if i.strip() != '']
        ex_no = 1
        for row in data:
            ex_st,ex_ed = [int(i) for i in row.split("-")]
            ex_sz = ex_ed - ex_st + 1 if ex_ed > ex_st else ex_st - ex_ed + 1
            result.append("%s,%s,%s,%s,%s"%(trans_id,ex_no,ex_st,ex_ed,ex_sz))
            ex_no += 1
        return result

    #final_data = 'sl_no,g_symbol,g_id,g_st,g_ed,strand,trans_id,exon_no,exon_st,exon_ed,exon_sz'
    final_data = ''
    for i in range(len(l_ind)):
        if i != len(l_ind) - 1:
            ps,pe = l_ind[i],l_ind[i+1]
            data_content = content[ps:pe]
            trans_id = [i for i in data_content[0].strip().split() if 'NM_' in i][0].replace(",","")
            #print (trans_id)
            t_data = get_transdata(data_content, trans_id)
            for d in t_data:
                r = f"{sl_no},{g_symbol},{g_id},{g_st},{g_ed},{strand},{d}"
                final_data += f"\n{r}"
                #print (r)
                sl_no += 1
        else:
            ps = l_ind[i]
            data_content = content[ps:]
            trans_id = [i for i in data_content[0].strip().split() if 'NM_' in i][0].replace(",","")
            #print (trans_id)
            t_data = get_transdata(data_content, trans_id)
            for d in t_data:
                r = f"{sl_no},{g_symbol},{g_id},{g_st},{g_ed},{strand},{d}"
                #print (r)
                final_data += f"\n{r}"
                sl_no += 1
        #print ("\n")
    #print (final_data)

    f = open(r"C:\Users\Admin\Desktop\PROJECT\Python\sqlworkbench\combined_gene.csv", 'a')
    f.write(final_data)
    f.close()
    
    print (filename, "DONE")

def create_database(dbname):
    connection = sql.connect(user = 'root', host = 'localhost', port = 3306, password = 'rahul@SQL')
    db = connection.cursor()
    query = 'create database if not exists %s'%(dbname)
    db.execute(query)
    connection.commit()
    db.close()
    connection.close()
    
def create_table(dbname, tbname):
    q = 'sl_no,g_symbol,g_id,g_st,g_ed,strand,trans_id,exon_no,exon_st,exon_ed,exon_sz'
    qt = ' varchar (100), '.join([i.strip() for i in q.split(",")]) 
    qt += ' varchar (100)'
    qt = f'create table if not exists {tbname} ({qt})'
    
    connection = sql.connect(user = 'root', host = 'localhost', port = 3306, database = dbname, password = 'rahul@SQL')
    db = connection.cursor()
    db.execute(qt) 
    connection.commit()
    db.close()
    connection.close()
    
def load_data(dbname, tbname):
    connection = sql.connect(user = 'root', host = 'localhost', port = 3306, database = dbname, password = 'rahul@SQL')
    db = connection.cursor()
    header = 'sl_no,g_symbol,g_id,g_st,g_ed,strand,trans_id,exon_no,exon_st,exon_ed,exon_sz'
    with open(r"C:\Users\Admin\Desktop\PROJECT\Python\sqlworkbench\combined_gene.csv", 'r') as fl:
        h = fl.readline().strip()
        check = 0
        while check == 0:
            dta = fl.readline().strip()
            
            if dta == '':
                check = 1;break;
                
            #print (dta)
            j = "','".join(dta.split(","))
            j = "'"+j+"'"
            q = f"insert into {tbname} (sl_no,g_symbol,g_id,g_st,g_ed,strand,trans_id,exon_no,exon_st,exon_ed,exon_sz) values ({j})"
            db.execute(q)
            
    connection.commit()
    db.close()
    connection.close()
    
if __name__ == "__main__":
    header = 'sl_no,g_symbol,g_id,g_st,g_ed,strand,trans_id,exon_no,exon_st,exon_ed,exon_sz'
    f = open(r"C:\Users\Admin\Desktop\PROJECT\Python\sqlworkbench\combined_gene.csv", 'w')
    f.write(header)
    f.close()
    
    file_list = os.listdir(r"C:\Users\Admin\Desktop\PROJECT\Python\sqlworkbench")
    file_list = [i for i in file_list if ".txt" in i]
    #print (file_list)
    for fn in file_list:
        print (fn, "started")
        g_data_to_csv(fn)
        #print ("\n")
    create_database('BIO_GENETIC_DATA')
    print ("Database Created")
    create_table("BIO_GENETIC_DATA", "gene_table")
    print ("Table Created")
    print ("Data uploading..........")
    load_data("BIO_GENETIC_DATA", "gene_table")
    print ("Data Uploaded completed.")
#print ("DONE")

g_data1.txt started
BRCA1 672 minus 43170327 43044295
g_data1.txt DONE
g_data2.txt started
TP53 7157 minus 7687490 7668421
g_data2.txt DONE
g_data3.txt started
MAD1L1 8379 minus 2232945 1815795
g_data3.txt DONE
Database Created
Table Created
Data uploading..........
Data Uploaded completed.
